# Sesión 3 de SM: Introducción al análisis de audio con Python

## Condigurar el repositorio de GitHub, conda y JupyterLab

* He creado un repositorio en github llamado practica2-SM y con el comando git clone lo clono como repostorio local para trabajar con el.
* Con el comando conda create --name=audioConda creo mi entorno de conda y con conda env list compruebo que esta bien creado y lo activo con conda activate.
* edito el .gitignore y añado la carpeta .ipynb_checkpoints/ para que la ignore.
* Usamos el comando conda install -c conda-forge <package_name> para instalar Python 3.10, ipykernel y JupyterLab.
* Añadimos el entorno de conda a los kernel de jupyterLab con python3 -m ipykernel install --user --name=<env_name>
* Instalamos jupyterLab en el entorno de conda conda install -c conda-forge jupyterlab
* Por ultimo Ejecutamos jupyterLab con el comando jupyter-lab y creamos un notebook vacio usando como kernel el entorno de conda "audioconda" creado anteriormente

## Análisis de audio con Python y jupyterLab

### Importar librerias y modulos de python

Importamos las librerias y modulos que vamos a utilizar de la siguiente forma

In [ ]:
#import librosa
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt #incluimos esta libreria para poder crear las graficas en el dominio del tiempo

### Especificamos los directorios de entrada salida

Aqui definiremos los directorios que ulitizaremos para almacenar los ficheros de audio con los que vamos a trabajar y donde almacenaremos los ficheros de audio que generaremos a lo largo de la práctica

In [ ]:
#directorio que utilizaremos
cwd = os.getcwd()
entrada_audio = os.path.join(cwd, os.path.join('audio', '_entrada'))#guardamos en la variable el path de la carpeta _entrada
salida_audio = os.path.join(cwd, os.path.join('audio', '_salida'))#guardamos en la variable el path de la carpeta _salida
print(f'Directorio con los audios de entrada: {entrada_audio}')
print(f'Directorio donde guardaremos los audios generados: {salida_audio}\n')

### Cargar del archivo de audio estereo y mostrar sus caracteristicas

Utilizaremos un archivo de audio .wav en este caso

In [ ]:
#cargamos el archivo de audio
filename = os.path.join(entrada_audio, 'game_of_thrones.wav')
#Mostrar informacion (sonido estéreo)
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}')#Mostramos el numero de filas y columnas de la matriz
print(f'- 1º canal:   {audio_data[:10, 0]}...')#Mostramos los 10 primeros elementos del vector correspondiente al canal 0 
print(f'- 2º canal:   {audio_data[:10, 1]}...')#Mostramos los 10 primeros elementos del vector correspondiente al canal 1
print(f'- Resolucion: {type(audio_data[0,0])}\n')

Vamos a escuchar el audio

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate)# .T se pasa únicamente si es audio estéreo.

### Convertir el archivo de audio estereo a mono

Vamos a calcular por simplificacion la media por canal para obtener un sonido mono

In [ ]:
# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  #Hace la media de los dos canales de audio y convierte el audio de estereo a mono
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')
print(f'- Canal unico:  {new_data_mono[:5]}...')
#Mantenemos la misma resolucion que en el audio estereo
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')

Ahora guardaremos el archivo y lo reproduciremos


In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav utilizando wavfile.write .
wavfile.write(
    filename=os.path.join(salida_audio, 'game_of_thrones_mono.wav'),#guardamos el archivo de audio convertido a mono en la carpeta _salida
    rate=sample_rate,
    data=new_data_mono
)
#Añadimos el widget para reproducir el audio
IPython.display.Audio(new_data_mono, rate=sample_rate)

Mostramos el tamaño de ambos archivos pra ver la difrencia

In [ ]:
!ls -sh audio/_entrada/game_of_thrones.wav
!ls -sh audio/_salida/game_of_thrones_mono.wav

### Difrencia entre audio estéreo y mono 

El audio estereo tiene dos canales de audio distinto porque los que sale audio difrente, proporcionando una experiencia mas inversiva, mientras que el audio mono al ser la media de los dos canales estereos todo el sonido sale por el mismo canal, lo que produce que salga el mismo sonido por los dos altavoces de los auriculares.

# Sesión 4 de SM: Procesamiento de audio con Python

### Grafica en el dominio del tiempo para el audion mono y estereo

Creamos la funcion que nos permite generar la grafica a en el dominio del tiempo.

In [ ]:
#Funcion para mostrar la grafica en el domino del tiempo del audio mono
def plot_mono_waveform(audio, sr, title):
    time = np.arange(0, len(audio)) / sr #generamos el eje de tiempo en segundos
    plt.figure(figsize =(12,6)) #se crea una nueva figura de tamaño 10x4
    plt.plot(time, audio) #dibujamos la forma la onda de audio en el tiempo
    plt.title(title) #nombre de la grafica
    plt.xlabel("Tiempo (s)") #etiqueta del eje x
    plt.ylabel("Amplitud") # etiqueta del eje y
    plt.grid(True)
    plt.show() #mostramos la grafica

#funcion para mostrar la grafica en el dominio del tiempo del audio estereo
def plot_stereo_waveform(audio_left, audio_right, sr, title):
    time = np.arange(0, len(audio_left)) / sr #generamos el eje de tiempo en segundos
    plt.figure(figsize =(12,6)) #se crea una nueva figura de tamaño 10x4
    plt.plot(time, audio_left, label="canal izquierdo") #dibujamos la forma la onda de audio en el tiempo del canal izquierdo
    plt.plot(time, audio_right, label="canal derecho") #dibujamos la forma la onda de audio en el tiempo del canal derecho
    plt.title(title) #nombre de la grafica
    plt.xlabel("Tiempo (s)") #etiqueta del eje x
    plt.ylabel("Amplitud") # etiqueta del eje y 
    plt.legend()
    plt.grid(True)
    plt.show() #mostramos la grafica

Ahora se cargan los archivos de audio y se generan sus respectivas graficas

In [ ]:
#cargamos el archivo de audio mono
filename_mono = os.path.join(salida_audio, 'game_of_thrones_mono.wav')
sr_mono, audio_mono = wavfile.read(filename_mono)
#Mostramos la grafica para el audio mono
plot_mono_waveform(audio_mono, sr_mono, "Audio Mono")

#cargamos el archivo de audio estereo
filename_estereo = os.path.join(entrada_audio, 'game_of_thrones.wav')
sr_stereo, audio_stereo = wavfile.read(filename_estereo)
#extraemos los canales izquierdo y derecho del audio estereo
audio_left = audio_stereo[:,0]
audio_right = audio_stereo[:,1]
#mostramos la grafica para el audio estereo
plot_stereo_waveform(audio_left, audio_right, sr_stereo, "Audio Estereo")



### Definiciones de frecuencia de muestreo, aliasing, profundidad de bits, ancho de banda y tasa de bits

* Frecuencia de muestreo: Es la velocidad a la que se toman muestras del audio o tambien se puede definir como el numero de muestras que se toman por segundo, a mayor frecuencia de muestreo mayor sera la calidad de una muestra.
* Aliasing: Es el nombre que se le da al efecto que provoca que dos señales continuas distintas se vuelvan indistinguibles al muestrearlas digitalmente a causa de una frecuencia de muestreo demasiado baja.
* Profundidad de bits: Hace referencia a la cantidad de bits disponible para medir la onda sonora y para su posterior almacenamiento en bytes digitales.
* Ancho de Banda: Se obtiene de la medicion conjunta de la profundidad de bits y la frecuencia de muestreo, por lo tanto el ancho de banda define la precision de nuestra señal digital con respecto a la grabacion original.
* Tasa de bits: Es el cálculo matemático del tamaño de los archivos digitales en megabytes por segundo(Mbps), en base a esto se puede decir que determina el número de bits que el ordenador debe procesar por segundo para reproducir la grabación de audio digital de la forma prevista.
  

### Aplicacion de la trasformada rapida de Fourier(FFT) para cambiar al dominio de la frecuencia

Caragamos el archivo de audio mono al que vamos a aplicar la FFT y llamamos a las distintas funciones para calcular la transformada

In [ ]:
sr_mono, audio_data_mono = wavfile.read(filename=os.path.join(salida_audio, 'game_of_thrones_mono.wav'))

n = len(audio_data_mono)
Fs = sr_mono

#Calculando la transformada rapida de Fourier
ch_fourier = np.fft.fft(audio_data_mono)

#solo observamos las frecuencias por debajo de Fs/2
abs_ch_Fourier = np.absolute(ch_fourier[:n//2])

#hacemos la grafica
plt.figure(figsize =(8,6))
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()
                            

La transformada de fourier es un aherramienta fundamental en el procesamiento de señales y en particular en el analisis de señales de audio. La Transformada de Fourier se utiliza para descomponer una señal en sus componentes de frecuencia, permitiendo así analizar su contenido espectral. Esto nos permite identificar qué frecuencias están presentes en la señal y con qué amplitud, lo que es crucial para su análisis y procesamiento. Tambien nos permite aplicar operaciones de procesamiento de señales específicas, como filtros, amplificación o atenuación selectiva de ciertas frecuencias, modificación del espectro de frecuencia para efectos de audio, entre otros. 

### Calculo de energia del espectrograma y la frecuencia de corte

Ahora vamos a definir una frecuencia umbral $f_0$ por la que cortar el espectro, es decir, solo nos quedaremos con aquellas frecuencias que esten por debajo de este valor para el archivo de audio comprimido.

El parametro epsilon reoresenta la parte de la energia del espectro que no conservaremos.

In [ ]:
# Definimos diferentes epsilons.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

#seleccionamos un valor de la lista de epsilons
eps = eps[0]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para la energia del espectro 
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# calculamos la integral de la frecuencia que es igual a la energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara booleana que compara el valor de corte con la energia del espectro e indica que frecuencias deben ser eliminadas
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# Calculamos la frecuencia de corte f0 en Hz mltiplicando la cantidad de frecuencias que deben ser eliminadas por el espacio de frecuencias y normalizando segun la longitud de la señal
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2) 
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Generamos la grafica.
plt.figure(figsize =(8,6))
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

Hemos usado el valor Epsilon: 1e-05 para reducir el tamaño del espectro dejando sin utilizar las frecuencias a partir de la frecuencia de corte f0 (Hz): 22008 y por lo tanto reduciendo el tamaño del archivo.

### Compresion de la onda aplicando downsampling

Vamos a proceder a reducir el tamaño del audio mono aplicando downsampling donde el factor de downsampling que vamos a utilizar se obtiene a partir de la frecuencia de corte anteriormente calculada

In [ ]:
wav_compressed_file = "game_of_thrones_mono_compressed.wav"

# Calculamos el factor D de downsampling dividiendo la frecuencia de muestreo entre la frecuencia de corte
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos 
new_data = audio_data_mono[::D]

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(salida_audio, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

#Cargamos el nuevo archivo
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(salida_audio, wav_compressed_file))
old_sample_rate, old_audio_data = wavfile.read(os.path.join(salida_audio, 'game_of_thrones_mono.wav'))

### Espectrograma

Aqui realizaremos el espectrograma de ambas ondas para ver las difrencias

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(old_audio_data, NFFT=1024, Fs=old_sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

Se puede apreciar como la resolucion se ha reducido, aunque en ciertas secciones de la onda se siguen apreciando caracteristicas similares.

Vamos a escuchar las dos pistas de audio, la comprimida y la original para compararlas 

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

In [ ]:
IPython.display.Audio(old_audio_data, rate=old_sample_rate)

Se puede apreciar claramente que en la pista comprimida los instrumentos que suenan de fondo en la melodia dejan de oirse bien

Despues de la compresion como se puede comprobar el tamaño es menor

In [ ]:
!ls -sh audio/_salida/game_of_thrones_mono.wav
!ls -sh audio/_salida/game_of_thrones_mono_compressed.wav